In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/commonlitreadabilityprize/train.csv')
train.head()

In [ ]:
test_df = pd.read_csv('/kaggle/input/commonlitreadabilityprize/test.csv')
test_df.head()

In [ ]:
print('Size of train data is ',train.shape)
print('Size of test data is ',test_df.shape)

In [ ]:
from fastai.data.all import *
from fastai.text.all import *

In [ ]:
text_lm = DataBlock(blocks=TextBlock.from_df('excerpt', is_lm=True),
                    get_x=ColReader('text'),
                    splitter=RandomSplitter())

dls_lm = text_lm.dataloaders(train, bs=64, seq_len=72)
dls_lm.show_batch(max_n=6)

In [ ]:
text_clas = DataBlock(blocks=(TextBlock.from_df('excerpt', seq_len=72, vocab=dls_lm.vocab), RegressionBlock),
                      get_x=ColReader('text'),
                      get_y=ColReader('target'),
                      splitter=RandomSplitter())

dls = text_clas.dataloaders(train, bs=64)
dls.show_batch()

In [ ]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=rmse,pretrained=True)

In [ ]:
learn.fit_one_cycle(1, 2e-2)
learn.save('1epoch')

In [ ]:
tdl = learn.dls.test_dl(test_df.excerpt)

In [ ]:
preds,_ = learn.get_preds(dl=tdl)

In [ ]:
ss = pd.read_csv('/kaggle/input/commonlitreadabilityprize/sample_submission.csv')
ss['target'] = preds.numpy()
ss.to_csv('submission.csv',index=False)